In [1]:
!pip install opencv-python opencv-contrib-python

In [ ]:
import cv2
import numpy as np
import time

# Load SSD model (MobileNet-SSD)
prototxt_path = "C:/Users/menak/CLG/CV/MobileNetSSD_deploy.prototxt"
caffemodel_path = "C:/Users/menak/CLG/CV/MobileNetSSD_deploy.caffemodel"
net = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)

# Define object classes relevant to self-driving cars
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat", "bottle", "bus",
           "car", "cat", "chair", "cow", "diningtable", "dog", "horse", "motorbike",
           "person", "pottedplant", "sheep", "sofa", "train", "tvmonitor"]
RELEVANT_CLASSES = {"car", "bus", "motorbike", "person", "bicycle", "traffic light"}

# Load video file (set video path or use 0 for webcam)
video_source = "C:/Users/menak/CLG/CV/FREE STOCK FOOTAGE - Heavy traffic.mp4"  # Replace with 0 for webcam
cap = cv2.VideoCapture(video_source)

if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

last_frame = None  # Store the last processed frame
last_capture_time = time.time()  # Initialize time tracking

while True:
    ret, frame = cap.read()
    if not ret:
        break  # Stop if video ends

    current_time = time.time()
    
    # Capture frame only if 2 seconds have passed
    if current_time - last_capture_time >= 2:
        (h, w) = frame.shape[:2]
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 0.007843, (300, 300), 127.5)
        net.setInput(blob)
        detections = net.forward()

        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.4:  # Confidence threshold
                idx = int(detections[0, 0, i, 1])
                label = CLASSES[idx]

                if label in RELEVANT_CLASSES:
                    box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                    (startX, startY, endX, endY) = box.astype("int")

                    # Draw bounding box and label
                    cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
                    text = f"{label}: {confidence * 100:.1f}%"
                    cv2.putText(frame, text, (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                    print(f"Detected: {label} with {confidence * 100:.1f}% confidence")

        cv2.imshow("Self-Driving Car Object Detection", frame)
        last_frame = frame.copy()  # Store last frame
        last_capture_time = current_time  # Reset time tracker

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

# Show the last captured frame with detections
if last_frame is not None:
    cv2.imshow("Final Detected Frame", last_frame)
    cv2.waitKey(0)  # Wait indefinitely until key is pressed
    cv2.destroyAllWindows()